In [1]:
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation

from explaneat.core.neuralneat import NeuralNeat as NeuralNeat

import random

import torch
import torch.nn as nn
import torch.optim as optim

import neat

from explaneat.core.utility import MethodTimer


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
# from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter

from explaneat.core.neuralneat import NeuralNeat as nneat

import torch.optim as optim

from torch.functional import F








In [2]:
RANDOM_SEED      = 42
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)


USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


In [3]:

def xor(a, b, threshold = 0.5):
    response = False
    if a > threshold and b < threshold:
        response = True
    if a < threshold and b > threshold:
        response = True
    # return (1.0, 0.0) if response else (0.0, 1.0)
    return 1.0 if response else 0.0
    

def create_n_points(n, size, min=0.0, max=1.0):
    data = []
    for _ in range(n):
        data.append([
            random.uniform(min, max) for ii in range(size)
        ])

    return data

# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

def overUnder(val, threshold):
    return 1. if val > threshold else 0

xor_inputs = create_n_points(400, 2, -1, 1)

xor_outputs = [
    [xor(tup[0], tup[1], 0)] for tup in xor_inputs
]

In [4]:
config_path = "./config_xor"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    with MethodTimer("Backprop everything"):
        p = BackpropPopulation(config, 
                                xs, 
                                ys, 
                                criterion=nn.BCEWithLogitsLoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

def eval_genomes(genomes, config):
    
    print(genomes)
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes.items():
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xor_inputs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(xor_outputs)))

config = base_config
saveLocation = './'
maxNGenerations = 2
p = instantiate_population(config, xor_inputs, xor_outputs, saveLocation)
# Run for up to nGenerations generations.
winner = p.run(eval_genomes, maxNGenerations, nEpochs = 5)

g = p.best_genome


The function - Backprop everything - has just started at 1644195753.132503
The function - Backprop everything - took 0.0025060176849365234 seconds to complete
The function - generationStart - has just started at 1644195753.135274

 ****** Running generation 0 ****** 

The function - generationStart - took 3.0040740966796875e-05 seconds to complete
The function - pre_backprop - has just started at 1644195753.135435
The function - pre_backprop - took 8.392333984375e-05 seconds to complete
The function - backprop - has just started at 1644195753.1355321
about to start backprop with 5 epochs
mean improvement: -0.0016957445211506583
best improvement: -0.002604102747592596
best loss: 0.5006431066353582
The function - backprop - took 0.2758960723876953 seconds to complete
The function - post_backprop - has just started at 1644195753.411478
The function - post_backprop - took 5.1021575927734375e-05 seconds to complete
The function - evaluate fitness - has just started at 1644195753.41167
{1: <

In [5]:
print(g)

Key: 106
Fitness: 1.3626459836959839
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.1238014040242528, response=1.0, activation=sigmoid, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=-2.594966385433642, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=-0.02214345565287014, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=-0.455489692522227, enabled=False)
	DefaultConnectionGene(key=(-1, 1), weight=1.6524593780898431, enabled=True)
	DefaultConnectionGene(key=(1, 0), weight=-1.082095303594005, enabled=True)


In [6]:
net = nneat(g, p.config, criterion=nn.BCEWithLogitsLoss())


In [7]:
net.genome.connections
# Error is in inputs not being updated with extra info from skip layers

{(-1, 0): <neat.genes.DefaultConnectionGene at 0x7fcbe0151df0>,
 (-2, 0): <neat.genes.DefaultConnectionGene at 0x7fcbe0151e50>,
 (-1, 1): <neat.genes.DefaultConnectionGene at 0x7fcbe0151f70>,
 (1, 0): <neat.genes.DefaultConnectionGene at 0x7fcbe0151fd0>}

In [8]:
net.forward(torch.tensor( xor_inputs[:10], dtype=torch.float64) ) [:10]

tensor([[0.4459],
        [0.4629],
        [0.4296],
        [0.4164],
        [0.4596],
        [0.4613],
        [0.4683],
        [0.4393],
        [0.4604],
        [0.4285]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [9]:
net.forward(torch.tensor( xor_inputs, dtype=torch.float64) )[:20]

tensor([[0.4459],
        [0.4629],
        [0.4296],
        [0.4164],
        [0.4596],
        [0.4613],
        [0.4683],
        [0.4393],
        [0.4604],
        [0.4285],
        [0.4215],
        [0.4615],
        [0.4037],
        [0.4684],
        [0.4171],
        [0.4210],
        [0.4431],
        [0.4557],
        [0.4194],
        [0.4155]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [10]:
# net.optimiser.zero_grad()

In [11]:
# net.optimise(xor_inputs, xor_outputs)

In [12]:
optimiser = optim.Adadelta(net.parameters())

In [13]:
net.weights

{2: Parameter containing:
 tensor([[ 0.0000],
         [-0.0221],
         [-1.0821]], dtype=torch.float64, requires_grad=True),
 1: Parameter containing:
 tensor([[1.6525],
         [0.0000]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}

In [14]:
optimizer = optimiser
losses = []
for i in range(10000):
    preds = net.forward(torch.tensor( xor_inputs, dtype=torch.float64) )
    loss = F.mse_loss(preds, torch.tensor( xor_outputs, dtype=torch.float64)).sqrt()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses.append(loss)  
losses[-10:]


[tensor(0.3110, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3110, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3110, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3110, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3110, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3110, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3110, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3110, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3110, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.3110, dtype=torch.float64, grad_fn=<SqrtBackward0>)]

In [15]:
preds[:10]

tensor([[0.4537],
        [0.1222],
        [0.1333],
        [0.8947],
        [0.0720],
        [0.6354],
        [0.0056],
        [0.8048],
        [0.7977],
        [0.7745]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [16]:
net.weights

{2: Parameter containing:
 tensor([[  5.9992],
         [  6.1189],
         [-17.7367]], dtype=torch.float64, requires_grad=True),
 1: Parameter containing:
 tensor([[3.2649],
         [3.1909]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}

In [17]:
net.layers

{2: {'nodes': {0: {'depth': 2,
    'output_ids': [],
    'input_ids': [-1, -2, 1],
    'depths': [],
    'output_layers': [],
    'needs_skip': False,
    'id': 0,
    'input_layers': [0, 0, 1],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': True,
  'is_input_layer': False,
  'layer_type': 'OUTPUT',
  'input_layers': [0, 1],
  'input_shape': 3,
  'weights_shape': (3, 1),
  'out_weights': [],
  'bias': [-0.1238014040242528],
  'in_weights': [[0]],
  'input_weights': array([[ 0.        ],
         [-0.02214346],
         [-1.0820953 ]]),
  'input_map': {(-2, 0): (1, 0), (1, 0): (2, 0)}},
 1: {'nodes': {1: {'depth': 1,
    'output_ids': [0],
    'input_ids': [-1],
    'depths': [],
    'output_layers': [2],
    'needs_skip': False,
    'id': 1,
    'input_layers': [0],
    'skip_layer_input': False,
    'layer_index': 0}},
  'is_output_layer': False,
  'is_input_layer': False,
  'layer_type': 'CONNECTED',
  'input_layers': [0],
  'input_shape': 2,
  'weights_sh

In [18]:
net.node_tracker[0]['depth']

2

In [19]:


reached_nodes = []

node_stack = []

# Instantiate stack with depth==0 nodes
for node_id, node in net.node_tracker.items():
    if node_id < 0:
        node_stack.append(node_id)

while len(node_stack) > 0:
    reached_nodes.append(node_stack[0])
    for node_id in net.node_tracker[node_stack[0]]['output_ids']:
        node_stack.append(node_id)
    del(node_stack[0])

if True:
    print("the reached nodes are")
    print(reached_nodes)


for node_id in net.node_tracker:
    if not node_id in reached_nodes:
        print(reached_nodes)
        print(net.node_tracker)
        print("I can't reach this node {}".format(node_id))

the reached nodes are
[-1, -2, 0, 1, 0, 0]


In [20]:
print(net.genome)

Key: 106
Fitness: 1.3626459836959839
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.1238014040242528, response=1.0, activation=sigmoid, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=-2.594966385433642, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=-0.02214345565287014, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=-0.455489692522227, enabled=False)
	DefaultConnectionGene(key=(-1, 1), weight=1.6524593780898431, enabled=True)
	DefaultConnectionGene(key=(1, 0), weight=-1.082095303594005, enabled=True)


In [21]:
net.node_mapping.layers

{2: {'nodes': {0: {'depth': 2,
    'output_ids': [],
    'input_ids': [-2, 1],
    'depths': [1, 2],
    'on_path_to_output': True,
    'on_path_to_input': False,
    'is_input': False,
    'is_output': True,
    'is_valid': True,
    'output_layers': [],
    'input_layers': [0, 1],
    'skips_in': True,
    'skips_out': False,
    'layer': 2,
    'layer_index': 0}},
  'is_output_layer': True,
  'is_input_layer': False,
  'layer_type': 'OUTPUT',
  'layer_activation': 'Sigmoid',
  'input_layers': [0, 1],
  'input_shape': 3,
  'output_shape': 1,
  'weights_shape': (3, 1),
  'input_map': {},
  'shape': (3, 1)},
 1: {'nodes': {1: {'depth': 1,
    'output_ids': [0],
    'input_ids': [-1],
    'depths': [1],
    'on_path_to_output': True,
    'on_path_to_input': True,
    'is_input': False,
    'is_output': False,
    'is_valid': True,
    'output_layers': [2],
    'input_layers': [0],
    'skips_in': False,
    'skips_out': False,
    'layer': 1,
    'layer_index': 0}},
  'is_output_layer':

In [22]:
for layer_id in sorted (net.node_mapping.layers.keys()):
    layer = net.node_mapping.layers[layer_id]
    print(layer['shape'])

(0, 2)
(2, 1)
(3, 1)
